Analysis Titan's winning 921 blocks.

Author: Bryanzk
Date Modified: 2024.04.11
License: CC0

In [1]:
from data_prep_utils import *
df = get_block_data_with_winning_bids()
dft = get_titan_won_921_blocks_bidding_data()
df_builder_info = get_builder_info()

In [3]:
# Filter out the duplicate block_number records with the earliest bid_timestamp, which is the winning bid thru the fastest relay
# N.B.: this would omit the records with the same block_number but different bid_timestamp from differe relays
df_filtered = df.copy()
df_filtered = df_filtered.sort_values(['block_number', 'bid_timestamp_ms'])
df_filtered = df_filtered.drop_duplicates('block_number', keep='first')

# get the unique block numbers won by Titan
unique_block_numbers_won_by_titan = dft['block_number'].unique()

df_filtered_titan = df_filtered[df_filtered['block_number'].isin(unique_block_numbers_won_by_titan)]
# df_filtered_titan.loc[:,'bid_timestamp_ms'] = pd.to_datetime(df_filtered['bid_timestamp_ms'], unit='ms')

df_filtered_titan.loc[:,'bid_timestamp_ms'] = df_filtered_titan['bid_timestamp_ms'].astype('int64') # 确保时间戳为整数
df_filtered_titan.loc[:,'bid_timestamp_ms'] = pd.to_datetime(df_filtered_titan['bid_timestamp_ms'], unit='ms')


# 计算时间差并且存储在新的 Dataframe 中 Calculate the time difference and store it in a new Dataframe.
ts_diff_df_titan = (df_filtered_titan['block_datetime'] - df_filtered_titan['bid_timestamp_ms']).apply(lambda x: x.total_seconds()) * 1000

# 添加新的列到原始的 Dataframe 中.  Add a new column, ts_diff, as ms difference, to the original Dataframe.
# if ts_diff > 0, bid before 12s, if ts_diff<0, bid after 12s
df_filtered_titan = pd.concat([df_filtered_titan, ts_diff_df_titan.rename('ts_diff')], axis=1)
